In [2]:
import pandas as pd
import numpy as np

In [3]:
header = ['user_id', 'movie_id', 'rating', 'timestamp']
df_ratings = pd.read_csv('Data/u.data', sep='\t', names=header)
df_ratings.head()

user_id  movie_id  rating  timestamp
0      196       242       3  881250949
1      186       302       3  891717742
2       22       377       1  878887116
3      244        51       2  880606923
4      166       346       1  886397596

In [4]:
movie_details = df_ratings.groupby('movie_id').agg({'rating':
                                                   [np.size, np.mean]})
movie_details.head()

rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

In [5]:
cols = ['movie_id', 'title']
#latin-1西歐語言編碼
movies = pd.read_csv('Data/u.item', sep='|', names=cols, usecols=range(2), encoding='latin-1')
ratings_titles = pd.merge(movies, df_ratings).sort_values(['rating'], ascending=False)
ratings_titles.head()

movie_id                                   title  user_id  rating  \
33615       218                        Cape Fear (1991)      388       5   
53455       357  One Flew Over the Cuckoo's Nest (1975)      532       5   
53463       357  One Flew Over the Cuckoo's Nest (1975)      499       5   
31674       208               Young Frankenstein (1974)      643       5   
17608       125                       Phenomenon (1996)      109       5   

       timestamp  
33615  886441083  
53455  892519935  
53463  885599372  
31674  891448136  
17608  880564534

In [55]:
#試試相似度衡量來做影片名稱關聯推薦 #Funtime
list(ratings_titles.title[0:3])

['Cape Fear (1991)',
 "One Flew Over the Cuckoo's Nest (1975)",
 "One Flew Over the Cuckoo's Nest (1975)"]

In [47]:
def Bag_of_Words(raw_corpus):  
    import jieba
    from sklearn.feature_extraction.text import CountVectorizer
    corpus = [] 
    for x in raw_corpus:
        corpus.append(x)
        #corpus.append(' '.join(jieba.cut(x)))
    vectorizer = CountVectorizer()
    corpus_matrix = vectorizer.fit_transform(corpus)
    return corpus_matrix.toarray()

In [58]:
x1,x2,x3 = Bag_of_Words(list(ratings_titles.title[0:3]))

In [60]:
def cosine_similarity(x1,x2):    
    import math
    sum_x_and_x, sum_x_and_y, sum_y_and_y = 0, 0, 0
    for i in range(len(x1)):
        x = x1[i]
        y = x2[i]
        sum_x_and_x += x*x
        sum_y_and_y += y*y
        sum_x_and_y += x*y
    return sum_x_and_y/math.sqrt(sum_x_and_x * sum_y_and_y)
print(cosine_similarity(x1,x3))

0.0


## Similarity matrix

In [6]:
#透過rating做相似度矩陣
ratings_ = df_ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
ratings_.head(5)

movie_id  1     2     3     4     5     6     7     8     9     10    ...   \
user_id                                                               ...    
1          5.0   3.0   4.0   3.0   3.0   5.0   4.0   1.0   5.0   3.0  ...    
2          4.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   2.0  ...    
3          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...    
4          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...    
5          4.0   3.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...    

movie_id  1673  1674  1675  1676  1677  1678  1679  1680  1681  1682  
user_id                                                               
1          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
3          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
5          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[5 rows x 1682 columns]

In [8]:
matrix = ratings_.as_matrix()
matrix 

array([[ 5.,  3.,  4., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 5.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  5.,  0., ...,  0.,  0.,  0.]])

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
ratings_train, ratings_test = train_test_split(matrix, test_size=0.3, random_state=42)

In [12]:
ratings_train.shape

(660, 1682)

In [13]:
ratings_test.shape

(283, 1682)

## Finding the top N nearest neighbors

In [14]:
from sklearn.neighbors import NearestNeighbors
k = 5

In [15]:
knn = NearestNeighbors(k, 'cosine')

In [16]:
knn.fit(ratings_train)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius='cosine')

In [18]:
top_k_dist, top_k_users = knn.kneighbors(ratings_train, return_distance=True)

In [25]:
top_k_dist.shape
top_k_users.shape

(660, 5)

## Top 5, similar users to user 1:

In [30]:
top_k_users[0][:5] #接近的user

array([  0, 211,  16, 583, 428])

In [31]:
top_k_dist[0][:5]

array([  0.        ,  41.71330723,  43.3474336 ,  45.04442252,  45.04442252])

In [32]:
#Prediction matrix for rating movies:
#為每個用戶選擇前5位用戶並使用他們的評級信息
#同時使用所有的加權和來預測評級
#前5名類似用戶的評分。

In [33]:
user_pred = np.zeros(ratings_train.shape)
for i in range(ratings_train.shape[0]):
    user_pred[i,:] = top_k_dist[i].T.dot(ratings_train[top_k_users][i])/np.array([np.abs(top_k_dist[i].T).sum(axis=0)]).T

user_pred.shape

(660, 1682)

In [34]:
df = pd.DataFrame(user_pred, columns=ratings_.columns)
df.head(10)

movie_id      1         2         3         4     5     6         7     \
0         0.000000  0.000000  0.000000  0.000000   0.0   0.0  0.000000   
1         0.000000  0.000000  0.000000  0.000000   0.0   0.0  0.000000   
2         0.000000  0.000000  0.000000  0.000000   0.0   0.0  0.000000   
3         0.000000  0.000000  0.000000  0.000000   0.0   0.0  0.000000   
4         2.753043  0.499327  0.249663  1.500827   0.0   0.0  1.753989   
5         0.000000  0.000000  0.000000  0.000000   0.0   0.0  0.000000   
6         4.000000  0.000000  0.000000  0.000000   0.0   0.0  2.733508   
7         0.000000  0.000000  0.000000  0.000000   0.0   0.0  0.000000   
8         0.000000  0.000000  0.000000  0.000000   0.0   0.0  0.000000   
9         3.758043  2.251767  0.000000  2.759742   0.0   0.0  3.496350   

movie_id      8         9     10    ...   1673  1674  1675  1676  1677  1678  \
0         0.000000  0.000000   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   
1         0.000000  0.000000   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   
2         0.000000  0.000000   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   
3         0.000000  1.230096   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   
4         0.499327  0.753749   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   
5         0.000000  0.000000   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   
6         0.000000  0.767198   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   
7         0.000000  0.000000   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   
8         0.000000  0.000000   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   
9         0.743135  0.748233   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   

movie_id  1679  1680  1681  1682  
0          0.0   0.0   0.0   0.0  
1          0.0   0.0   0.0   0.0  
2          0.0   0.0   0.0   0.0  
3          0.0   0.0   0.0   0.0  
4          0.0   0.0   0.0   0.0  
5          0.0   0.0   0.0   0.0  
6          0.0   0.0   0.0   0.0  
7          0.0   0.0   0.0   0.0  
8          0.0   0.0   0.0   0.0  
9          0.0   0.0   0.0   0.0  

[10 rows x 1682 columns]

## Evaluating the model:

In [30]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [31]:
def rmse(pred,actual):
    prediction = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, actual))

In [32]:
rmse(user_pred, ratings_train)

2.860210336806541

In [33]:
rmse(user_pred, ratings_test)

3.367330398504572

## Item-based recommendations

In [36]:
# defining k as the number of movies
k_ = ratings_train.shape[1]
k_

1682

In [37]:
neigh_ = NearestNeighbors(k_, 'cosine')

In [38]:
neigh_.fit(ratings_train.T)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=1682, p=2,
         radius='cosine')

In [39]:
top_k_dist_, top_k_movies_ = neigh_.kneighbors(ratings_train.T, return_distance=True)

## Similar movies to movie 1:

In [40]:
top_k_movies_[0] #選第0個電影，有可能相近的電影有誰

array([  0, 120, 404, ..., 285, 312, 301])

In [41]:
top_k_dist_[0] #距離

array([  0.        ,  53.12249994,  54.38749856, ...,  77.08436936,
        77.39509028,  79.3473377 ])

## Predicting the movie ratings

In [42]:
item_predictions = ratings_train.dot(top_k_dist_) / np.array([np.abs(top_k_dist_).sum(axis=1)])

In [44]:
df_predictions = pd.DataFrame(item_predictions, columns=ratings_.columns)
df_predictions.head(10)
#Because the query set matches the training set, the nearest neighbor of each point is the point itself, at a distance of zero.

movie_id  1         2         3         4         5         6         7     \
0          0.0  0.318426  0.370093  0.262237  0.369791  0.462891  0.188307   
1          0.0  0.054538  0.062280  0.044045  0.061896  0.077263  0.031369   
2          0.0  0.035225  0.040731  0.028824  0.040595  0.050637  0.020564   
3          0.0  0.087291  0.101434  0.071960  0.101578  0.126977  0.051637   
4          0.0  0.421538  0.488463  0.347368  0.490747  0.614978  0.250371   
5          0.0  0.062268  0.071446  0.050520  0.071088  0.088917  0.036149   
6          0.0  0.127591  0.149610  0.105822  0.149263  0.186985  0.076077   
7          0.0  0.047315  0.054840  0.038746  0.054376  0.067811  0.027574   
8          0.0  0.038242  0.044113  0.031148  0.043863  0.054703  0.022212   
9          0.0  0.521024  0.606827  0.431687  0.609992  0.763399  0.310718   

movie_id      8         9         10      ...         1673      1674  \
0         0.240255  0.208729  0.352278    ...     1.298808  1.292362   
1         0.039994  0.034737  0.058565    ...     0.182118  0.180352   
2         0.026209  0.022772  0.038423    ...     0.166232  0.165331   
3         0.065800  0.057136  0.096375    ...     0.331993  0.330009   
4         0.319556  0.277829  0.469281    ...     1.790325  1.779009   
5         0.046140  0.040088  0.067561    ...     0.205173  0.203762   
6         0.097033  0.084321  0.142406    ...     0.449655  0.446358   
7         0.035161  0.030520  0.051402    ...     0.156799  0.155842   
8         0.028351  0.024608  0.041484    ...     0.132831  0.131897   
9         0.396555  0.344655  0.581956    ...     2.121418  2.108141   

movie_id      1675      1676      1677      1678      1679      1680  \
0         1.365190  1.380232  1.393514  1.408556  1.426800  1.448504   
1         0.189936  0.191750  0.193745  0.195906  0.197830  0.200787   
2         0.174985  0.177631  0.179116  0.180139  0.181806  0.184569   
3         0.347356  0.351866  0.354874  0.358003  0.360961  0.365639   
4         1.881473  1.902814  1.922278  1.941175  1.968537  1.997288   
5         0.214561  0.216829  0.219402  0.221169  0.223249  0.225526   
6         0.470311  0.475094  0.478856  0.482460  0.487072  0.491807   
7         0.164196  0.165506  0.166585  0.168679  0.170884  0.172929   
8         0.138646  0.140299  0.141655  0.142829  0.144261  0.146185   
9         2.229936  2.254356  2.276204  2.297575  2.326097  2.360383   

movie_id      1681      1682  
0         1.456640  1.545308  
1         0.201121  0.213325  
2         0.186547  0.200763  
3         0.367251  0.390919  
4         2.013134  2.126436  
5         0.226530  0.240815  
6         0.490226  0.521446  
7         0.171849  0.185213  
8         0.145816  0.156544  
9         2.365945  2.521556  

[10 rows x 1682 columns]

In [42]:
df_predictions.shape

(660, 1682)

## Evaluating the model:

In [43]:
rmse(item_predictions, ratings_train)

3.342538761948114

In [44]:
rmse(item_predictions, ratings_test)

3.44626980973761